In [ ]:
# Connecting with third party MySQL data storage files.io

!pip install mysql-connector-python


import mysql.connector
from mysql.connector import Error

hostname = "" # Add the hostname of the dataset from the files.io
database = "" # Add the database name of the dataset from the files.io
port = "" # Add the port number from the files.io
username = "" # Add the username of the dataset from the files.io
password= "" # Add the password of the dataset from the files.io

try:
    connection = mysql.connector.connect(host=hostname, database=database, user=username, password=password, port=port)
    if connection.is_connected():
      db_Info = connection.get_server_info()
      print("Connected to MySQL Server version ", db_Info)
      cursor = connection.cursor()
      cursor.execute("select database();")
      record = cursor.fetchone()
      print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 61.0 MB/s eta 0:00:00


/tmp/ipython-input-3479796774.py:16: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_Info = connection.get_server_info()


Connected to MySQL Server version  8.0.36-28
You're connected to database:  ('olistproject_personalus',)
MySQL connection is closed


In [ ]:
import pandas as pd

order_payments = pd.read_csv("olist_order_payments_dataset.csv") # Reading the csv file called olist_payment_dataset.csv
order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [ ]:

import pandas as pd
import mysql.connector
from mysql.connector import Error

# Connection details (Credentials are availabe on files.io once dataset is there)
hostname = ""
database = ""
port = ""
username = ""
password = ""

# CSV file path
csv_file_path = "olist_order_payments_dataset.csv"

# Table name where the data will be uploaded
table_name = "olist_order_payments"


try:
  # Step 1: Establish a connection to MySQL server
  connection = mysql.connector.connect(
    host=hostname,
    database=database,
    user=username,
    password=password,
    port=port
  )
  if connection.is_connected():
      print("Connected to MySQL Server successfully!")

      # Step 2: Create a cursor to execute SQL queries
      cursor = connection.cursor()

      # Step 3: Drop table if it already exists (for clean insertion)
      cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
      print (f"Table {table_name} dropped if it existed.")

      # Step 4: Create a table structure to match CSV file
      create_table_query = f"""
      CREATE TABLE {table_name} (
        order_id VARCHAR(50),
        payment_sequential INT,
        payment_type VARCHAR(20),
        payment_installments INT,
        payment_value FLOAT
      );
      """

      cursor.execute(create_table_query)
      print(f"Table {table_name} created successfully!")


      # Step 5: Load the CSV data into pandas DataFrame
      data = pd.read_csv(csv_file_path)
      print("CSV data loaded into pandas DataFrame.")


      # Step 6: Inserting data in batches of 1000 records
      batch_size = 1000 # Define the batch size
      total_records = len(data) # Get total records in the DataFrame

      print(f"Starting data insertion into `{table_name}` in batches of {batch_size} records.")
      for start in range (0, total_records, batch_size):
          end = start + batch_size
          batch = data.iloc[start:end] # Get the current batch of records

          # Converting batch to list of tuples for MySQL insertion
          batch_records=[
              tuple(row) for row in batch.itertuples (index=False, name=None)
          ]

          # Preparing the INSERT query to push data in relational data form
          insert_query = f"""
          INSERT INTO {table_name}
          (order_id, payment_sequential, payment_type, payment_installments, payment_value)
          VALUES (%s, %s, %s, %s, %s);
          """
          # Executing the insertion query for the batch
          cursor.executemany (insert_query, batch_records)
          connection.commit() # Commit after each batch
          print(f"Inserted records {start + 1} to {min (end, total_records)} successfully.")

      print(f"All {total_records} records inserted successfully into '{table_name}`.")


except Error as e:
    # Step 7: Handling any errors
    print("Error while connecting to MySQL or inserting data:", e)
finally:
    # Step 8: Closingthe cursor and connection
    if connection.is_connected():
      cursor.close()
      connection.close()
      print("MySQL connection is closed.")












Connected to MySQL Server successfully!
Table olist_order_payments dropped if it existed.
Table olist_order_payments created successfully!
CSV data loaded into pandas DataFrame.
Starting data insertion into `olist_order_payments` in batches of 1000 records.
Inserted records 1 to 1000 successfully.
Inserted records 1001 to 2000 successfully.
Inserted records 2001 to 3000 successfully.
Inserted records 3001 to 4000 successfully.
Inserted records 4001 to 5000 successfully.
Inserted records 5001 to 6000 successfully.
Inserted records 6001 to 7000 successfully.
Inserted records 7001 to 8000 successfully.
Inserted records 8001 to 9000 successfully.
Inserted records 9001 to 10000 successfully.
Inserted records 10001 to 11000 successfully.
Inserted records 11001 to 12000 successfully.
Inserted records 12001 to 13000 successfully.
Inserted records 13001 to 14000 successfully.
Inserted records 14001 to 15000 successfully.
Inserted records 15001 to 16000 successfully.
Inserted records 16001 to 17

In [ ]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 25.1 MB/s eta 0:00:00


In [ ]:
#importing module
from pymongo import MongoClient

# Connection details (Credentials are availabe on files.io once dataset is there)
hostname = ""
database=""
port = ""
username = ""
password = ""
uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

# Connecting with the portnumber and host
client = MongoClient(uri)

# Accessing database
mydatabase = client [database]

In [ ]:

import pandas as pd
from pymongo import MongoClient

# Load the product_category CSV file into a pandas DataFrame
try:
  product_category_df = pd.read_csv("product_category_name_translation.csv")
except FileNotFoundError:
  print("Error: 'product_category_name_translation.csv' not found.")
  exit() # Exit the script if the file is not found


# MongoDB connection details (User can get it from files.io)
hostname = ""
database = ""
port = ""
username = ""
password = ""

uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

try:
    # Establishing a connection to MongoDB
    client = MongoClient(uri)
    db = client[database]

    # Selecting the collection (or create if it doesn't exist)
    collection = db["product_categories"]  # Choose a suitable name for your collection

    # Converting the DataFrame to a list of dictionaries for insertion into MongoDB
    data_to_insert = product_category_df.to_dict(orient="records")

    # Inserting the data into the collection
    collection.insert_many(data_to_insert)

    print("Data uploaded to MongoDB successfully!")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Closing the MongoDB connection
    if client:
        client.close()

Data uploaded to MongoDB successfully!
